>*SYNTASA - Clean-Up of Telegraph.co.uk HITlog*

<center><h2>Code to Clean-Up Variables in the HITlog</h2><center><br>
<center><h3>Out-of-Box Adobe - Syntasa Variables - Custom Metrics</h3><center>
<center><h3>BATCH 2 : Visitor Identifiers & Info</h3><center>

><p style="color:teal; font-style:bold"><b>Field: customer_code</b>
><p style="color:purple; font-style:bold"><b>Label: evar9_PID</b>
- A visitor ID passed in with the hit to be used as the visitor ID for the hit (instead of using visid_high and visid_low)
- customer_code is always NULL as we do not use it. 
- post_evar9 is the Telegraphs unique identifier (cusotmer ID from cookie tmg_pid) that is given to any digital registrants or subscribers
- Recommendation: Use customer_code as a slot for the PID. <b>Source post_evar9</b>
- Note on occasion this pulls in the TS number instead but there is nothing I can do about this. 

><p style="color:teal; font-style:bold"><b>Field: visitor_id</b>
><p style="color:purple; font-style:bold"><b>Label: cookie</b>
- Visitor_Id is the Cookie
- Cookie is a concatenation of post_visid_high & post_visid_low
- Recommendation:  Rename this to "Cookie" (no change to code)

In [ ]:
--cookie
CONCAT(post_visid_high, post_visid_low)

><p style="color:teal; font-style:bold"><b>Field: mcvisid</b>
><p style="color:purple; font-style:bold"><b>Label: mcvisid</b>

><p style="color:teal; font-style:bold"><b>Field: cust_field352</b>
><p style="color:purple; font-style:bold"><b>Label: visitor_id_method</b>
- <b>M</b>arketing <b>C</b>loud <b>Vis</b>itor <b>ID</b>
- The Marketing Cloud Visitor ID is a 128-bit number that consists of 2 signed, positive, 64-bit numbers in base 10, both padded to 19 zeros, and then concatenated.
- This returns as '00000000000000000000000000000000000000' when an mcvisid is not deployed
- Since April 2018 the number of Cookies with no mcvisid has been creeping up (probably GDPR related)
- April 2018 (6.67% of uniques), May 2018 (7.93% of uniques), June 2018 (8.46% of uniques)
- Recommendation: Leave as it is. Use Cookie instead of mcvisid as the uniques identifier. Syntasa have not captured <b>post_visid_type</b> which is a good variable to pull in to give some idea of why the mcvisid may not have been deployed. The majority of cookies with post_visid_page = 3 or 5 have an associated mcvisid. Add this variable. User friendly code for this post_visid_type is provided below:

In [ ]:
CASE 
WHEN post_visid_type IS NULL THEN NULL
WHEN post_visid_type = '0' THEN 'vid (s.visitorID) - custom method'
WHEN post_visid_type = '1' THEN 'Does not accept Cookies'
WHEN post_visid_type = '2' THEN 'HTTP Mobile header - No Cookies Mobile Device'
WHEN post_visid_type = '3' THEN 'aid (s_vi.cookie) - existing cookie or grace period'
WHEN post_visid_type = '4' THEN 'fid (fallback cookie) - 1st party but no mcvisid'
WHEN post_visid_type = '5' THEN 'mid (AMCV_ cookie set by ExpCloudID service)'
ELSE 'Unassigned'
END
--visitor_id_method

><p style="color:teal; font-style:bold"><b>Field: is_new_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: is_first_hit</b>
><p style="color:teal; font-style:bold"><b>Field: cust_field353</b>
><p style="color:purple; font-style:bold"><b>Label: is_new_visitor</b>
- is_new_visitor(last_hit_time_gmt)
- The logic for is_new_visitor returns 1 (or New) when last_hit_time_gmt (which is the timestamp of the previous HIT) is 0 or NULL (i.e. when it is their very first hit) or returns zero for any other value. 
- This logic means that a visitor will only be classed as new for their first HIT and not their first Session. 
- So this logic actaully generates a variable more like <b>is_first_hit</b>
- Recommendation: visit_num is a more reliable variable than last_hit_time_gmt because the latter can show as 0 when the data is messy. When visit num is >=1 then last_hit_time_gmt becomes more reliable. However, we only need to use both to determine whether the hit is the very first hit ever. The plan is to create 2 variables: is_first_hit and is_new_visitor using the combination of visit_num and last_hit_time_gmt. 
- Instead of a boolean the output should be a string with the options: 
- <b>FirstHit, notFirstHit, Unknown</b> - for is_first_hit
- <b>New, Repeat, Unknown</b> - for is_new_visitor

In [ ]:
CASE 
WHEN visit_num IS NULL THEN 'Unknown' 
WHEN visit_num = '0' THEN 'Unknown' 
WHEN visit_num > '1' THEN 'notFirstHit'
WHEN visit_num = '1' AND last_hit_time_gmt = '0' THEN 'FirstHit'     
WHEN visit_num = '1' AND last_hit_time_gmt != '0' THEN 'notFirstHit'
ELSE 'Unassigned'
END
--is_first_hit,

In [ ]:
CASE 
WHEN visit_num IS NULL THEN 'Unknown' 
WHEN visit_num = '0' THEN 'Unknown' 
WHEN visit_num > '1' THEN 'Repeat' 
WHEN visit_num = '1' THEN 'New'
ELSE 'Unassigned'
END
--is_new_visitor,

><p style="color:maroon; font-style:bold"><b>Removed Field: is_repeat_visitor</b>
- This variable is superfluous as the same data is collected in the is_new_visitor variable. 
- Recommendation: Remove the source for this variable so that it doesn't show up in the schema

><p style="color:teal; font-style:bold"><b>Field: cust_field9</b>
><p style="color:purple; font-style:bold"><b>Label: evar10_visitor_type</b>
- evar10 returns 3 pieces of information about the visitor, whether they are: (i) New or Repeat (ii) LoggedIn or Out (iii) Anonymous, Registered or Subscribed
- Recommendation: Pull out just the Anon, Sub, Reg part of this using the case statement below:

In [ ]:
CASE
WHEN ( LOWER(post_evar10) LIKE '%registered%' ) THEN 'Registered'
WHEN ( post_evar10 LIKE '%in;R%' OR post_evar10 LIKE 'out%;R' ) THEN 'Registered'
WHEN ( LOWER(post_evar10) LIKE '%subscribe%' ) THEN 'Subscribed'
WHEN post_evar10 LIKE '%in;S' THEN 'Subscribed'
WHEN ( LOWER(post_evar10) LIKE '%anonym%') THEN 'Anonymous'
WHEN post_evar10 LIKE '%out;A%' THEN 'Anonymous'
ELSE 'Unassigned'
END

><p style="color:teal; font-style:bold"><b>Field: last_seen_gmt</b>
><p style="color:purple; font-style:bold"><b>Label: last_seen_datetime</b>
- The field "last_seen_gmt" is passed by default with Adobe Clickstream data sources.
- In Adobe it is called last_hit_time_gmt
- This is the timestamp of the previous hit in GMT
- This "timestamp" is in Unix time (number of seconds since Jan 1, 1970.)											
- Recommendation: Use the code below to convert this into a more readable datetime format

In [ ]:
CASE 
WHEN last_hit_time_gmt = '0' THEN NULL
ELSE TIMESTAMP_SECONDS(SAFE_CAST(last_hit_time_gmt as int64))
END
--last_seen_datetime

><p style="color:teal; font-style:bold"><b>Field: last_seen</b>
><p style="color:purple; font-style:bold"><b>Label: last_seen_period</b>
- This function takes two timestamps, usually "hit_time_gmt" and "last_hit_time_gmt"
- It compares these two timestamps and returns one of the following options which is written to the row: 
    1. "less than 1 day"
    2. "1 to 3 days"
    3. "3 to 7 days" 
    4. "7 to 14 days"
    5. "14 days to 1 month" 
    6. "Longer than 1 month"
    7. "Unknown" 										
- Recommendation: Leave as it is for now but check it is working after the implementaions. Rename to <b>last_seen_period</b>

><p style="color:teal; font-style:bold"><b>Field: hourly_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: hourly_visitor</b>
- A flag to determine if current hit is a new hourly visitor
- The first HIT for a visitor that hour of the day is given 1 and every subsequent HIT in the hour is assigned 0
- This variable can be problematic as a hour is defined by the time zone of the report suite (in our case UK)
- Any session that spans midnight (more likely for non-UK visitors) will be split and the visitor will be counted as a daily visitor in both days. 
- Recommendation: Leave as it is

><p style="color:teal; font-style:bold"><b>Field: daily_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: daily_visitor</b>
- A flag to determine if current hit is a new daily visitor.
- The first HIT for a visitor that day is given 1 and every subsequent HIT is assigned 0
- This variable can be problematic as a day is defined by the time zone of the report suite (in our case UK)
- A session that spans midnight (more likely for non-UK visitors) will be split and the visitor will be counted as a daily visitor in both days. 
- Recommendation: This variable is actually useful to determine where sessions have been split between tables and for that reason is worth keeping. This happens for sessions WHERE daily_visitor = 1 AND visit_page_num != 1

><p style="color:teal; font-style:bold"><b>Field: weekly_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: weekly_visitor</b>
- A flag to determine if current hit is a new weekly visitor
- The first HIT for a visitor that week is given 1 and every subsequent HIT in the week is assigned 0
- This variable can be problematic as a week is defined by the time zone of the report suite (in our case UK)
- Recommendation: Leave as it is

><p style="color:teal; font-style:bold"><b>Field: monthly_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: monthly_visitor</b>
- A flag to determine if current hit is a new monthly visitor
- The first HIT for a visitor that month is given 1 and every subsequent HIT in the month is assigned 0
- Month is defined by the time zone of the report suite (in our case UK)
- This will be useful to determine whether a visitor has been active in the month which can be used to determine monthly activity in the customer table
- Recommendation: Leave as it is

><p style="color:teal; font-style:bold"><b>Field: quarterly_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: quarterly_visitor</b>
- A flag to determine if current hit is a new quarterly visitor.
- The first HIT for a visitor that quarter is given 1 and every subsequent HIT in the quarter is assigned 0
- Recommendation: I'm not sure how useful this is givent that we have a monthly flag. I would rather do a rolling 90days activity in the Customer table. Leave as it is for now but consider removing. 

><p style="color:teal; font-style:bold"><b>Field: yearly_visitor</b>
><p style="color:purple; font-style:bold"><b>Label: yearly_visitor</b>
- A flag to determine if current hit is a new yearly visitor
- The first HIT for a visitor that year is given 1 and every subsequent HIT in the year is assigned 0
- This variable can be problematic as a week is defined by the time zone of the report suite (in our case UK)
- Recommendation: Leave as it is

><p style="color:teal; font-style:bold"><b>Field: cust_field77</b>
><p style="color:purple; font-style:bold"><b>Label: evar77_tmgconnect_registered</b>
- Specifies Telegraph Connect Registered User Status
- Telegraph Connect is a network of communities dedicated to helping professionals grow their businesses and develop their careers
- Once captured for a browser this evar never expires
- Returns NULL is the browser is not Registered for Telegraph Connect otherwise it specifies whhether the browser is Business Regstered or Media Registered. 
- RECOMMENDATION: Use the code below to make the NULL value more readable. 

In [ ]:
CASE
WHEN post_evar77 IS NULL THEN 'Not Telegraph Connect Registered'
ELSE post_evar77
END
--evar77_tmgconnect_registered